# Testing BertPipeline

In [2]:
import sys
sys.path.append('../ml/BERT')
import masking
import BERT
from Vectorisation import Vectorisation
from Config import Config

import pickle
import tensorflow as tf
import numpy as np
with open("../../data/ml4science_data.pkl", "rb") as fp:
    data_dict = pickle.load(fp)

config = Config(EPOCH=100)
vec = Vectorisation(config=config)

In [3]:
with open('../../data/sequences.pkl', 'rb') as f:
    decoded_sequences = pickle.load(f)

for i in range(3): print(decoded_sequences[0][i])
print(len(decoded_sequences))

[0, 0, 0, 1, 0, 0, 0, 0, 0, 7.889]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 75.141, 0.0]
[0.0, 0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 3.25]
254


### Vectorising sequences

In [4]:
seps = vec.sep_from_seq(decoded_sequences)
encoded_sequences = vec.encode(decoded_sequences, seps)

print(encoded_sequences.shape)
print(encoded_sequences[0])

(254, 128)
[26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
  3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
  9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


### Masking sequences, split into train and test
x -> masked  
y -> unmasked

In [5]:
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(encoded_sequences, config.TOKEN_DICT)
x_tr, y_tr, w_tr = x_masked_encoded[:200], y_masked_encoded[:200], sample_weights[:200]
x_te, y_te, w_te = x_masked_encoded[200:], y_masked_encoded[200:], sample_weights[200:]

print('full set:', x_masked_encoded.shape, y_masked_encoded.shape, sample_weights.shape)
print('train set:', x_tr.shape, y_tr.shape, w_tr.shape)
print('test set:', x_te.shape, y_te.shape, w_te.shape)

# this is the dataset that will be used for training
mlm_ds = tf.data.Dataset.from_tensor_slices((x_tr, y_tr, w_tr))

full set: (254, 128) (254, 128) (254, 128)
train set: (200, 128) (200, 128) (200, 128)
test set: (54, 128) (54, 128) (54, 128)


In [6]:
from BERTPipeline import BERTPipeline

bert = BERTPipeline(config)
bert.train(mlm_ds)


Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 word_embedding (Embedding)  (None, 128, 64)              1728      ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 128, 64)              0         ['word_embedding[0][0]']      
 Lambda)                                                                                          
                                                                                                  
 encoder_0/multiheadattenti  (None, 128, 64)              16640     ['tf.__operat

### Predicting on the masked test set

In [7]:
pred = bert.predict(x_te)

2/2 [==============================] - 0s 7ms/step


In [8]:
print(pred.shape)
print('pred:', pred[19])
print('test:', y_te[19])
print((pred[19] == y_te[19])*1)

(54, 128)
pred: [25 26 23 21 21 22 26 21 21 26 22 22 22 22 26 23 26 26 26 26 26 19 19 19
 19 16 16 19 17 17 19 19 19 19 19 19 19 16  7  7  7 17 17 17 17  7  7  7
 16 16 16 16 16 16  7  7  7  7 16 16 16  5  7  7  7  7 20 20  7  7 16 15
 15 15 15 15  4  4  4  8  8  9  9 15 14 14  4  4 17 20 20 15  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  5  5  5  5  4  4  4  9  9  9  9 11  4  4
  4  4  4  4  4  4  4  4]
test: [25 26  2 21 21  2 26 21 21 26 25 22 22 22 26 23 26 25  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [9]:
where_equal = (pred == y_te)
where_masked = (x_te == config.TOKEN_DICT['[MASK]'])

print("Accuracy brut (per student mean): ", np.mean(np.sum(pred == y_te, axis=1) / (y_te.shape[1])))
print("Accuracy brut (per student mean): ", np.sum(pred == y_te) / np.size(y_te))
print("Accuracy without padding (per student mean): ", np.mean(np.sum((pred == y_te) * (y_te != 0), axis=1) / np.sum(y_te != 0, axis=1)))
print("Accuracy on masked tokens (per student mean): ", np.mean(np.sum(where_equal * where_masked, axis=1) / np.sum(where_masked, axis=1)))

Accuracy brut (per student mean):  0.4017650462962963
Accuracy brut (per student mean):  0.4017650462962963
Accuracy without padding (per student mean):  0.747873078073344
Accuracy on masked tokens (per student mean):  nan


C:\Users\yanni\AppData\Local\Temp\ipykernel_22284\566778706.py:7: RuntimeWarning: invalid value encountered in divide
  print("Accuracy on masked tokens (per student mean): ", np.mean(np.sum(where_equal * where_masked, axis=1) / np.sum(where_masked, axis=1)))


In [11]:
print("Accuracy brut (per total): ", np.sum(pred == y_te) / np.size(y_te))
print("Accuracy without padding (per total): ", np.sum((pred == y_te) * (y_te != 0)) / np.sum(y_te != 0))
print("Accuracy on masked tokens: ", np.sum(where_equal * where_masked) / np.sum(where_masked))

Accuracy brut (per total):  0.4017650462962963
Accuracy without padding (per total):  0.749527665317139
Accuracy on masked tokens:  0.38155136268343814
